In [3]:
%load_ext autoreload
%autoreload 2

import cProfile as cp
import pstats
import numpy as np
import meerkat as mk
import pandas as pd
import torch
import spacy

import os
os.environ["TERRA_CONFIG_PATH"] = "/Users/sabrieyuboglu/code/domino/terra_config.json"
import terra

In [38]:
from domino.data.celeb import build_celeb_df, build_celeb_dp
#df = build_celeb_df("/Users/sabrieyuboglu/data/datasets/celeba")
df = build_celeb_df.out().load()
dp = build_celeb_dp(df)
index = np.random.randint(1000)

In [39]:
print(len(dp.data._block_refs))
%timeit out = dp.lz[500:100000]

45
46.1 ms ± 613 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
dp.consolidate()

In [41]:
print(len(dp.data._block_refs))
%timeit out = dp.lz[500:100000]

1
42.4 ms ± 440 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
df = pd.DataFrame(
    np.random.randn(1000000, 10)
)

In [29]:
df[0] is df[0]

True

In [13]:
%timeit df.iloc[500:50000]

16.5 µs ± 977 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [15]:
dp = mk.DataPanel.from_pandas(df)
dp.remove_column("index")

In [16]:
print(len(dp.data._block_refs))
%timeit out = dp.lz[500:50000]

10
1.03 ms ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [18]:
dp.consolidate()

In [22]:
out = dp.lz[500:50000]

In [31]:
data = np.zeros(10)

In [32]:
%timeit [mk.NumpyArrayColumn(data) for i in range(100)]

1.39 ms ± 9.95 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
import pstats
profiler = cp.Profile()
profiler.enable()
dp.lz[500:50000]
profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats

In [2]:
dp = mk.DataPanel({
    f"{idx}": mk.NumpyArrayColumn(np.arange(1000000))
    for idx in range(100)
})
dp.remove_column("index")

In [3]:
%timeit dp.lz[100:50000]

1.08 ms ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [4]:
dp.consolidate()

In [6]:
%timeit dp.lz[100:50000]

371 µs ± 989 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
import pstats
profiler = cp.Profile(timeunit=.00001)
profiler.enable()
dp.lz[500:50000]
profiler.disable()
stats = pstats.Stats(profiler)
stats.sort_stats('cumtime').print_stats(20)

         6998 function calls (6897 primitive calls) in 0.004 seconds

   Ordered by: cumulative time
   List reduced from 80 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.004    0.002 /Users/sabrieyuboglu/miniconda3/envs/domino/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3396(run_code)
        2    0.000    0.000    0.004    0.002 {built-in method builtins.exec}
        1    0.000    0.000    0.004    0.004 <ipython-input-11-622a4f1b2e88>:4(<module>)
        1    0.000    0.000    0.004    0.004 /Users/sabrieyuboglu/code/meerkat/meerkat/mixins/materialize.py:14(__getitem__)
        1    0.000    0.000    0.004    0.004 /Users/sabrieyuboglu/code/meerkat/meerkat/datapanel.py:480(_get)
  201/101    0.000    0.000    0.003    0.000 /Users/sabrieyuboglu/code/meerkat/meerkat/mixins/cloneable.py:31(_clone)
        1    0.000    0.000    0.003    0.003 /Users/sabrieyuboglu/code/meerkat/me

In [33]:
x = mk.PandasSeriesColumn(np.arange(4))
x[np.array([True, False, True, False])]

TypeError: super(type, obj): obj must be an instance or subtype of type

In [40]:
col = pd.Series(np.arange(10))
index = torch.Tensor((col.values % 2).astype(bool))

In [46]:
col.iloc[index.to(bool)]

0    0
1    1
0    0
1    1
0    0
1    1
0    0
1    1
0    0
1    1
dtype: int64

In [65]:
x = pd.Series(np.arange(10)).to_frame()
y = x[0].iloc[:5]
x[0].values.base is y.values.base

True

In [64]:
x = pd.Series(np.arange(10))
y = x.iloc[:5]
x.values is y.values.base

True

In [62]:
x[0] is x[0]

True

In [7]:
pd.Series([1,2,3]).reset_index(drop=True)

0    1
1    2
2    3
dtype: int64

In [9]:
torch.ones(10).dtype

torch.float32

In [11]:
import numpy as np
from itertools import product

In [28]:
class NumpyArrayColumnTestBed:

    config = {
        "num_dims": [1,2,3],
        "dim_len": [1,5],
        "dtype": ["float", "int"],
    }

    def __init__(
        self, length: int=16, num_dims: int = True, dim_length: int = 5, dtype="float"
    ):
        np.random.seed(123)
        array = np.random.random(length, *[dim_length for _ in range(num_dims-1)]) * 10
        array = array.astype(dtype)
    
        self.column = mk.NumpyArrayColumn.from_array(array)
        self.data = array

    @classmethod
    def parameterize(cls, params: dict = None):
        return map(dict, product(*[[(k, v) for v in vs] for k, vs in cls.config.items()]))
       

In [29]:
out = NumpyArrayColumnTestBed.parameterize()

In [31]:
list(out)

[{'num_dims': 1, 'dim_len': 1, 'dtype': 'float'},
 {'num_dims': 1, 'dim_len': 1, 'dtype': 'int'},
 {'num_dims': 1, 'dim_len': 5, 'dtype': 'float'},
 {'num_dims': 1, 'dim_len': 5, 'dtype': 'int'},
 {'num_dims': 2, 'dim_len': 1, 'dtype': 'float'},
 {'num_dims': 2, 'dim_len': 1, 'dtype': 'int'},
 {'num_dims': 2, 'dim_len': 5, 'dtype': 'float'},
 {'num_dims': 2, 'dim_len': 5, 'dtype': 'int'},
 {'num_dims': 3, 'dim_len': 1, 'dtype': 'float'},
 {'num_dims': 3, 'dim_len': 1, 'dtype': 'int'},
 {'num_dims': 3, 'dim_len': 5, 'dtype': 'float'},
 {'num_dims': 3, 'dim_len': 5, 'dtype': 'int'}]